# Introduction

The concentration of air pollutants can depend on:
- Weather
- Traffic density
- Local factors (e.g., industry)

Although initially I decided to focus on weather and traffic density, I couldn't find a source that gave traffic data with a resolution better than yearly average. So in the end I used only weather data, past and forecast, the sources being:
- Historical: [Meteostat](https://meteostat.net/)
- Forecast bulk download: [World Weather Online](https://www.worldweatheronline.com/developer/) (free for 30 days)
- Forecast regular update: [Open Meteo](https://open-meteo.com/)

### Imports

In [1]:
import json
import numpy as np
import pandas as pd
import requests
import datetime as dt
import time
from dotenv import dotenv_values

# Meteostat (Historical actual weather data)

First we get the nearest weather stations to Montpellier and Marseille

In [2]:
# Import Meteostat library
from meteostat import Stations

In [3]:
city_coordinates = {
    'Montpellier' : (43.6106, 3.8738),
    'Marseille' : (43.2965, 5.3698)
}

In [4]:
# Get nearby weather stations
stations = Stations()
nearest_station_id = {}
for city, coords in city_coordinates.items():
    stations = stations.nearby(coords[0], coords[1])
    nearest_station_id[city] = stations.fetch(1).index[0]

Now we can get weather data for those stations

In [5]:
# Import Meteostat library and dependencies
from datetime import datetime
from meteostat import Hourly

# Set time period
start = datetime(2013, 1, 1)
end = datetime(2022, 8, 29, 23, 59)

hourly_data = {}
for city, station_id in nearest_station_id.items():
    hourly_data[city] = Hourly(station_id, start, end)

As an example we can see the data fetched for Montpellier.

In [6]:
hourly_data['Montpellier'].fetch()

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
time,,,,,,,,,,,
2016-01-01 00:00:00,10.1,9.3,95.0,NaN,NaN,10.0,9.4,NaN,1025.7,NaN,NaN
2016-01-01 01:00:00,10.1,9.3,95.0,0.0,NaN,10.0,7.6,NaN,1025.6,NaN,NaN
2016-01-01 02:00:00,10.2,9.4,95.0,0.0,NaN,30.0,5.4,NaN,1025.7,NaN,NaN
2016-01-01 03:00:00,10.1,9.3,95.0,NaN,NaN,20.0,7.6,NaN,1025.5,NaN,NaN
2016-01-01 04:00:00,10.2,9.4,95.0,0.0,NaN,40.0,11.2,NaN,1024.8,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2015-12-31 19:00:00,10.0,9.4,96.0,0.8,NaN,10.0,14.8,NaN,1026.1,NaN,NaN
2015-12-31 20:00:00,9.9,9.3,96.0,0.6,NaN,20.0,13.0,NaN,1026.1,NaN,NaN
2015-12-31 21:00:00,9.9,9.3,96.0,NaN,NaN,10.0,9.4,NaN,1026.1,NaN,NaN


Now we combine the data from both cities into a single dataframe.

In [7]:
hourly_data_dfs = {}
for city, df in hourly_data.items():
    hourly_data_dfs[city] = hourly_data[city].fetch().sort_index()
    hourly_data_dfs[city]['City'] = city

In [8]:
dfs_to_concat = [df for df in hourly_data_dfs.values()]
meteostat_weather_data = pd.concat(dfs_to_concat)
city_col = meteostat_weather_data.pop('City')
meteostat_weather_data.insert(0, 'City', city_col)

In [9]:
meteostat_weather_data

,City,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
time,,,,,,,,,,,,
2013-01-01 00:00:00,Montpellier,8.0,6.5,90.0,NaN,NaN,50.0,16.6,NaN,1017.1,NaN,NaN
2013-01-01 01:00:00,Montpellier,8.9,7.3,90.0,0.0,NaN,60.0,22.3,NaN,1015.7,NaN,NaN
2013-01-01 02:00:00,Montpellier,9.1,7.4,89.0,0.0,NaN,60.0,24.1,NaN,1015.2,NaN,NaN
2013-01-01 03:00:00,Montpellier,9.4,7.7,89.0,NaN,NaN,80.0,16.6,NaN,1014.8,NaN,NaN
2013-01-01 04:00:00,Montpellier,12.3,7.8,74.0,0.0,NaN,140.0,22.3,NaN,1014.1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-29 19:00:00,Marseille,25.9,23.2,85.0,0.0,NaN,180.0,7.6,16.7,1016.9,NaN,2.0
2022-08-29 20:00:00,Marseille,25.6,23.3,87.0,0.0,NaN,150.0,5.4,16.7,1017.2,NaN,2.0
2022-08-29 21:00:00,Marseille,24.7,22.6,88.0,0.0,0.0,130.0,7.6,11.0,1017.0,NaN,2.0


In [10]:
meteostat_weather_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 169205 entries, 2013-01-01 00:00:00 to 2022-08-29 23:00:00
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   City    169205 non-null  object 
 1   temp    169111 non-null  float64
 2   dwpt    169102 non-null  float64
 3   rhum    169102 non-null  float64
 4   prcp    127335 non-null  float64
 5   snow    17555 non-null   float64
 6   wdir    167108 non-null  float64
 7   wspd    169161 non-null  float64
 8   wpgt    74477 non-null   float64
 9   pres    168461 non-null  float64
 10  tsun    62 non-null      float64
 11  coco    79417 non-null   float64
dtypes: float64(11), object(1)
memory usage: 16.8+ MB


With only 62 non-null values the `tsun` column is not useful so we drop it

In [11]:
meteostat_weather_data.drop(columns = 'tsun', inplace = True)

In [12]:
meteostat_path = 'data/Historical weather/Historical to 2022-08-29/Meteostat historical data.gz'

In [13]:
meteostat_weather_data.to_csv(meteostat_path)

## WorldWeatherOnline (Forecast data)

### Bulk load

In [14]:
ENV_VARS = dotenv_values('.env.weather')

In [15]:
url = "https://api.worldweatheronline.com/premium/v1/past-weather.ashx"

wwo_querystring = {
    "key" : ENV_VARS['WWO_API_KEY'],
    "format" : "json",
    "tp" : "1",
    "includelocation" : "no"
}

In [16]:
def wwo_datetime_str(date_str, time_str):
    time_str = time_str.rjust(4, '0')
    return f"{date_str} {time_str[:2]}:{time_str[2:]}:00"

In [17]:
def get_wwo_hour_data_csv_str(date_str, hour_data, feature_cols):
    data = [hour_data[feature_col] for feature_col in feature_cols]
    return f"{wwo_datetime_str(date_str, hour_data['time'])},{','.join(data)}"

I chose the weather characteristics I thought likely to be of use.

In [18]:
wwo_feature_cols = [
    'tempC',
    'windspeedKmph',
    'winddirDegree',
    'winddir16Point',
    'precipMM',
    'humidity',
    'visibility',
    'pressure',
    'cloudcover',
    'HeatIndexC',
    'DewPointC',
    'WindChillC',
    'WindGustKmph',
    'FeelsLikeC',
    'uvIndex'
]

In [19]:
# Set csv headers
wwo_csv_str = 'location,datetime,' + ','.join(wwo_feature_cols)

We can only download one month at a time, so we run a loop to that for the period we want.

In [20]:
start_year, start_month = 2013, 1
end_year, end_month = 2022, 7

In [21]:
for location in ['Montpellier', 'Marseille']:

    wwo_querystring['q'] = location
    
    for year in range(start_year, end_year + 1):

        year_start_month = start_month if year == start_year else 1
        year_end_month = end_month if year == end_year else 12

        for month in range(year_start_month, year_end_month + 1):

            # Get first and last day in month.
            first_day_in_month = dt.datetime(year, month, 1)
            day_in_next_month = first_day_in_month.replace(day=28) + dt.timedelta(days=4)
            last_day_in_month = day_in_next_month - dt.timedelta(days=day_in_next_month.day)

            wwo_querystring['date'] = first_day_in_month.strftime('%Y-%m-%d')
            wwo_querystring['enddate'] = last_day_in_month.strftime('%Y-%m-%d')
            
            wwo_response = requests.request("GET", url, params = wwo_querystring)
            wwo_dict = json.loads(wwo_response.text)
            
            for day_data in wwo_dict['data']['weather']:
                date_str = day_data['date']
                for hour_data in day_data['hourly']:
                    wwo_csv_str += f"\n{location},{get_wwo_hour_data_csv_str(date_str, hour_data, wwo_feature_cols)}"
            
            time.sleep(1)
            print(year, month)

2013 1
2013 2
2013 3
2013 4
2013 5
2013 6
2013 7
2013 8
2013 9
2013 10
2013 11
2013 12
2014 1
2014 2
2014 3
2014 4
2014 5
2014 6
2014 7
2014 8
2014 9
2014 10
2014 11
2014 12
2015 1
2015 2
2015 3
2015 4
2015 5
2015 6
2015 7
2015 8
2015 9
2015 10
2015 11
2015 12
2016 1
2016 2
2016 3
2016 4
2016 5
2016 6
2016 7
2016 8
2016 9
2016 10
2016 11
2016 12
2017 1
2017 2
2017 3
2017 4
2017 5
2017 6
2017 7
2017 8
2017 9
2017 10
2017 11
2017 12
2018 1
2018 2
2018 3
2018 4
2018 5
2018 6
2018 7
2018 8
2018 9
2018 10
2018 11
2018 12
2019 1
2019 2
2019 3
2019 4
2019 5
2019 6
2019 7
2019 8
2019 9
2019 10
2019 11
2019 12
2020 1
2020 2
2020 3
2020 4
2020 5
2020 6
2020 7
2020 8
2020 9
2020 10
2020 11
2020 12
2021 1
2021 2
2021 3
2021 4
2021 5
2021 6
2021 7
2021 8
2021 9
2021 10
2021 11
2021 12
2022 1
2022 2
2022 3
2022 4
2022 5
2022 6
2022 7
2013 1
2013 2
2013 3
2013 4
2013 5
2013 6
2013 7
2013 8
2013 9
2013 10
2013 11
2013 12
2014 1
2014 2
2014 3
2014 4
2014 5
2014 6
2014 7
2014 8
2014 9
2014 10
2014 11
20

In [22]:
wwo_path = 'data/Weather forecast/Historical to 2022-08-29/WeatherWorldOnline forecast data to 29 Aug'

In [23]:
f = open(wwo_path + '.csv', "w+")
f.write(wwo_csv_str)
f.close()

### Specific date range

Since we have already downloaded the pollutants data up to 29 August, to make our data match up timewise we add on a few more days to get to 29 August

In [26]:
url = "https://api.worldweatheronline.com/premium/v1/past-weather.ashx"

wwo_querystring = {
    "key" : ENV_VARS['WWO_API_KEY'],
    "format" : "json",
    "tp" : "1",
    "date" : "2022-08-01",
    "enddate" : "2022-08-29",
}

In [27]:
additional_wwo_csv_str = ''

for location in ['Montpellier', 'Marseille']:
    
    wwo_querystring['q'] = location
    wwo_response = requests.request("GET", url, params = wwo_querystring)
    
    wwo_dict = json.loads(wwo_response.text)
    for day_data in wwo_dict['data']['weather']:
        date_str = day_data['date']
        for hour_data in day_data['hourly']:
            additional_wwo_csv_str += f"\n{location},{get_wwo_hour_data_csv_str(date_str, hour_data, wwo_feature_cols)}"

In [28]:
f = open(wwo_path + '.csv', "a")
f.write(additional_wwo_csv_str)
f.close()

In [30]:
wwo_data = pd.read_csv(wwo_path + '.csv', index_col = 'datetime')

In [31]:
wwo_data

,location,tempC,windspeedKmph,winddirDegree,winddir16Point,precipMM,humidity,visibility,pressure,cloudcover,HeatIndexC,DewPointC,WindChillC,WindGustKmph,FeelsLikeC,uvIndex
datetime,,,,,,,,,,,,,,,,
2013-01-01 00:00:00,Montpellier,11,19,139,SE,0.1,80,2,1017,84,11,8,9,31,9,1
2013-01-01 01:00:00,Montpellier,11,18,135,SE,0.1,81,2,1017,89,11,8,9,29,9,1
2013-01-01 02:00:00,Montpellier,11,17,131,SE,0.1,82,2,1016,95,11,8,9,27,9,1
2013-01-01 03:00:00,Montpellier,11,15,127,SE,0.2,83,2,1016,100,11,8,9,24,9,1
2013-01-01 04:00:00,Montpellier,11,13,107,ESE,0.1,83,2,1016,94,11,8,9,20,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-29 19:00:00,Marseille,27,19,137,SE,0.0,64,10,1014,7,29,20,27,24,29,1
2022-08-29 20:00:00,Marseille,27,17,136,SE,0.0,65,10,1014,6,28,19,27,22,28,1
2022-08-29 21:00:00,Marseille,26,15,136,SE,0.0,65,10,1014,5,28,19,26,21,28,1


In [32]:
wwo_data.index = pd.to_datetime(wwo_data.index)

In [33]:
wwo_data.sort_values(by = ['location', 'datetime'], ascending = [False, True], inplace = True)

In [34]:
wwo_data

,location,tempC,windspeedKmph,winddirDegree,winddir16Point,precipMM,humidity,visibility,pressure,cloudcover,HeatIndexC,DewPointC,WindChillC,WindGustKmph,FeelsLikeC,uvIndex
datetime,,,,,,,,,,,,,,,,
2013-01-01 00:00:00,Montpellier,11,19,139,SE,0.1,80,2,1017,84,11,8,9,31,9,1
2013-01-01 01:00:00,Montpellier,11,18,135,SE,0.1,81,2,1017,89,11,8,9,29,9,1
2013-01-01 02:00:00,Montpellier,11,17,131,SE,0.1,82,2,1016,95,11,8,9,27,9,1
2013-01-01 03:00:00,Montpellier,11,15,127,SE,0.2,83,2,1016,100,11,8,9,24,9,1
2013-01-01 04:00:00,Montpellier,11,13,107,ESE,0.1,83,2,1016,94,11,8,9,20,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-29 19:00:00,Marseille,27,19,137,SE,0.0,64,10,1014,7,29,20,27,24,29,1
2022-08-29 20:00:00,Marseille,27,17,136,SE,0.0,65,10,1014,6,28,19,27,22,28,1
2022-08-29 21:00:00,Marseille,26,15,136,SE,0.0,65,10,1014,5,28,19,26,21,28,1


In [35]:
wwo_data.to_csv(wwo_path + '.gz')

## Open Meteo (Weather forecast)

This is what we use to get future forecasts. Here is an example download for a period of 2022-09-12 to 2022-09-21. Note that I have chosen the weather characteristics that turned out to be of interest following feature analysis (shown in the notebook 'Feature analysis'.

In [36]:
city_coordinates = {
    'Montpellier' : (43.6106, 3.8738),
    'Marseille' : (43.2965, 5.3698)
}

In [37]:
open_meteo_responses = {}
forecast_hourly_data_dfs = []

In [38]:
url = 'https://api.open-meteo.com/v1/forecast'

weather_variables = [
    'temperature_2m',
    'relativehumidity_2m',
    'dewpoint_2m',
    'surface_pressure',
    'cloudcover',
    'windspeed_10m',
    'winddirection_10m',
]

url += '?hourly=' + ','.join(weather_variables)

querystring = {
    'timezone' : 'Europe/Paris',
    # Can set these to specific date but only about a month historical
    'start_date' : '2022-09-12', 
    'end_date' : '2022-09-21',
}

for city, coordinates in city_coordinates.items():
    querystring['latitude'] = str(coordinates[0])
    querystring['longitude'] = str(coordinates[1])
    
    response = requests.request("GET", url, params = querystring)
    open_meteo_responses[city] = json_data = json.loads(response.text)
    open_meteo_responses[city] = json.loads(response.text)
    forecast_hourly_data_df = pd.DataFrame(json_data['hourly'])
    forecast_hourly_data_df['location'] = city
    forecast_hourly_data_dfs.append(forecast_hourly_data_df)

In [40]:
forecast_hourly_data = pd.concat(forecast_hourly_data_dfs).rename(columns = {'time' : 'datetime'})

In [41]:
forecast_hourly_data['datetime'] = pd.to_datetime(forecast_hourly_data['datetime'])

In [42]:
forecast_hourly_data

,datetime,temperature_2m,relativehumidity_2m,dewpoint_2m,surface_pressure,cloudcover,windspeed_10m,winddirection_10m,location
0,2022-09-12 00:00:00,18.2,81,14.9,1008.4,0,3.5,336,Montpellier
1,2022-09-12 01:00:00,17.6,85,15.0,1008.3,0,3.9,326,Montpellier
2,2022-09-12 02:00:00,17.1,86,14.8,1008.8,0,4.2,329,Montpellier
3,2022-09-12 03:00:00,16.8,88,14.8,1008.3,0,4.2,340,Montpellier
4,2022-09-12 04:00:00,16.6,85,14.0,1007.9,84,4.7,351,Montpellier
...,...,...,...,...,...,...,...,...,...
235,2022-09-21 19:00:00,23.4,39,9.5,1016.7,43,12.9,117,Marseille
236,2022-09-21 20:00:00,22.6,45,10.5,1016.8,54,2.9,60,Marseille
237,2022-09-21 21:00:00,21.4,48,10.5,1018.0,50,4.6,39,Marseille
238,2022-09-21 22:00:00,20.2,52,10.6,1018.4,67,5.9,43,Marseille
